# Balance Metabolic Models

With Frowins Scripts I was able to obtain stoichiometric consistent models; next step is mostly to balance the charges

Workplan Martina:
* checken, dass alle eine chemische Formel haben (über Cobrapy)
* dann einmal ran an mass imbalances (falls es die gibt)
* charge imbalances
* Hier geht es vor allem darum in den verschiedenen Datenbanken (bigg, metacyc, kegg, etc). die richtigen Reaktionswege zu finden. Das ist der Teil in dem Protokoll, wo auch von den Protonenimbalances die Rede ist
* Ziel sollte am Ende sein, dass beim Aufruf der Reaktion (in Cobrapy): .check_mass_balance()  eine 0 (=Null) rauskommt, das heißt, dass die Reaktion keine Ladung hat.

## Imports & Paths

In [28]:
import cobra
import os
import subprocess
from cobra.io import read_sbml_model
from cobra.manipulation.validate import check_mass_balance

In [2]:
models_path = "Models/mass_balance/"

In [25]:
# import models after mass balancing through Frowins scripts
models = {}
for model_name in (f for f in os.listdir(models_path) if f.endswith(".xml")):
    model = read_sbml_model(f"{models_path}/{model_name}")
    model.solver = "cplex"
    models[model_name[:3]] = model

AA1, AA2, AA3, AA4, AA5, AA6, AA7 = [models[f"AA{i}"] for i in range(1, 8)]
# model_list = ["AA{i}" for i in range(1, 8)]

## Check Models

In [27]:
AA1

Name,AA1
Memory address,7cf0d2dc5990
Number of metabolites,1532
Number of reactions,2307
Number of genes,973
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [11]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(AA1, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

(<Model AA1 at 0x7cf0d6dfed10>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


In [13]:
# print("exchanges", AA1.exchanges)
print("demands", AA1.demands)
print("sinks", AA1.sinks)

demands []
sinks [<Reaction sink_2ohph_c at 0x7cf0cd913e50>, <Reaction sink_hemeO_c at 0x7cf0cdac7700>, <Reaction sink_mobd_c at 0x7cf0cd912f80>]


In [16]:
AA1.optimize()

,fluxes,reduced_costs
EX_13ppd_e,0.000000,0.000000e+00
EX_14glucan_e,-0.000000,0.000000e+00
EX_15dap_e,0.000000,0.000000e+00
EX_2ddglcn_e,-0.000000,0.000000e+00
EX_2dhglcn_e,-0.000000,0.000000e+00
...,...,...
sink_2ohph_c,-0.000000,0.000000e+00
sink_hemeO_c,-0.000000,0.000000e+00
sink_mobd_c,-0.000000,0.000000e+00
Growth,66.941934,-2.220446e-16


In [46]:
def check_charge_balance(model):
    unbalanced_reactions = check_mass_balance(model)
    print("There are {0} charge unbalanced reactions in {1}".format(len(unbalanced_reactions), model) )
    return unbalanced_reactions

In [49]:
AA1_charge_unbalances = check_charge_balance(models["AA1"])

There are 445 charge unbalanced reactions in AA1


Name,AA1
Memory address,7cf0d2dc5990
Number of metabolites,1532
Number of reactions,2307
Number of genes,973
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"
